In [ ]:
## 개별종목분석

from mod1 import *

df = select_stock('hrs','2019-01-01')
close_vol_ma(df,'ma120')

In [ ]:
##  거래량 증가 관심종목 분석

group_analysis(path_volume,'2019-09-27')

In [ ]:
##  가격 증가 관심종목 분석

group_analysis(path_price,'2019-09-30')